# COLLECTING DATA
---

Nhiệm vụ của phần này là thực hiện lấy dữ liệu từ thông tin imdbid và title trong file ids_titles.txt.

## 1. IMPORT THƯ VIỆN CẦN THIẾT

In [1]:
import pandas as pd
import requests
import re

## 2. THAY ĐỔI PHIÊN LÀM VIỆC

In [2]:
# chia 3 ra nha mina hong no k cho minh lay :D
# từ từ để ngâm cú chứ nhìn này trông cx hơi ngu :v
member_part = {
    "Trinh": {
        "part": 1,
        "api_key": None # đk api xong áp vô nè
    },

    "Thi": {
        "part": 2,
        "api_key": None # đk api xong áp vô nè
    },

    "MinhUyen": {
        "part": 3,
        "api_key": 'd4c83dec'
    } 
}

In [3]:
# Điền tên mn vào đây theo từ điển nè
info = member_part["MinhUyen"]

part = info["part"]
apiKey = info["api_key"]

## 3. ĐỌC DỮ LIỆU ĐƯỢC THU THẬP TRƯỚC ĐÓ ĐỂ TÌM KIẾM THÊM

In [4]:
file_name = f"../Data/pre_data_part{part}.csv"

In [5]:
pre_data = pd.read_csv(file_name)
pre_data.info() # kiểm tra có null kiểu nan không nha k là chet o 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 629 entries, 0 to 628
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id/Title      629 non-null    object
 1   Release year  629 non-null    int64 
 2   Age rating    629 non-null    object
 3   Runtime       629 non-null    object
 4   Genre         629 non-null    object
dtypes: int64(1), object(4)
memory usage: 24.7+ KB


## 4. HÀM LẤY DỮ LIỆU CỦA TRANG `OMDB API`

Hàm này trả về link từ id hoặc tiêu đề phim

In [6]:
def convert_url(item: str, year:int,  key = apiKey):
    if item.startswith("tt"):
        url = f'https://www.omdbapi.com/?i={item}&apikey={key}'
    else: 
        title = item.split(":")[-1].strip() # lấy tên sau :
        title = re.sub(r'[^a-zA-Z0-9\s]', '', title) # xóa kí tự đặc biệt
        title = '+'.join(title.split(' ')) # thêm dấu + giữa các từ
        url = f'https://www.omdbapi.com/?t={title}&y={year}&apikey={key}'

    return url

Hàm này thì thu thập dữ liệu

In [19]:
def collectMovieData(pre_data, key = apiKey):
    full_title = []
    full_rated = []
    released_time = []
    full_runtime = []
    full_genre = []
    metaScore = []
    rating = []
    vote = []
    boxOffice = []
    language = []
    award = []

    items = pre_data['Id/Title']
    release_year = pre_data['Release year']
    genre = pre_data['Genre']
    age_rating = pre_data['Age rating']
    runtime = pre_data['Runtime']

    for i in range(len(items)):
        print(f"[LOG]: Collecting data of {items[i]}")
        url = convert_url(items[i], year = release_year[i], key=key)
        trying = 1

        while True:    
            respond = requests.get(url)
            if respond.status_code == 200:
                data = respond.json()
                if data.get("Response") == "False": # bỏ qua những phim không thể tim thấy trong trang
                    break

                full_title.append(data.get('Title', 'N/A'))
                
                rated = age_rating[i]
                if rated == "Null":
                    rated = data.get('Rated', 'N/A')
                full_rated.append(rated)

                time = data.get('Released', 'N/A')
                if time == 'N/A':
                    time = release_year[i]
                released_time.append(time)

                run_time = runtime[i]
                if run_time == "Null":
                    run_time = data.get('Runtime', 'N/A')
                full_runtime.append(run_time)
                
                gen = data.get('Genre', 'N/A')
                if gen == 'N/A':
                    gen = genre[i]
                full_genre.append(gen)
                    
                metaScore.append(data.get('Metascore', 'N/A'))
                rating.append(data.get('imdbRating', 'N/A'))
                vote.append(data.get('imdbVotes', 'N/A'))
                boxOffice.append(data.get('BoxOffice', 'N/A'))
                language.append(data.get('Language', 'N/A'))
                award.append(data.get('Awards', 'N/A'))
                break
                
            else:
                trying += 1
                if trying > 3:
                    print(f"Request fail after 3 retries with code {respond.status_code}")
                import time
                time.sleep(2)

    movieDataframe = pd.DataFrame({
            "Title": full_title,
            "Release time": released_time,
            "Age rating": full_rated,
            "Runtime": full_runtime,
            "Genre": full_genre,
            "Lanuage": language,
            "Awards": award,
            "Metascore": metaScore,
            "IMDb Rating": rating,
            "IMDb Votes": vote,
            "Domestic Revenue": boxOffice
        })

    return movieDataframe

này là thu thập dữ liệu nè

In [20]:
df1 = collectMovieData(pre_data, key=apiKey)
df1

[LOG]: Collecting data of King of Clones
[LOG]: Collecting data of tt2560092
[LOG]: Collecting data of The Wiggles: The Sound of Christmas
[LOG]: Collecting data of tt19724192
[LOG]: Collecting data of tt7720142
[LOG]: Collecting data of tt20417104
[LOG]: Collecting data of Gen Hoshino Concert Recollections 2015-2023
[LOG]: Collecting data of tt28810493
[LOG]: Collecting data of A Place in the Field
[LOG]: Collecting data of Trading Places
[LOG]: Collecting data of tt3701150
[LOG]: Collecting data of tt18260564
[LOG]: Collecting data of tt11426538
[LOG]: Collecting data of tt18292986
[LOG]: Collecting data of tt27148236
[LOG]: Collecting data of tt26084136
[LOG]: Collecting data of tt15441472
[LOG]: Collecting data of tt23647954
[LOG]: Collecting data of Mafia Mamma
[LOG]: Collecting data of tt10338176
[LOG]: Collecting data of tt9201772
[LOG]: Collecting data of tt20833316
[LOG]: Collecting data of tt15460742
[LOG]: Collecting data of tt14230458
[LOG]: Collecting data of tt21156390
[L

,Title,Release time,Age rating,Runtime,Genre,Lanuage,Awards,Metascore,IMDb Rating,IMDb Votes,Domestic Revenue
0,King of Clones,23 Jun 2023,PG-13,1h 25min,Documentary,"English, Russian, Korean",1 nomination,N/A,6.1,954,N/A
1,The Magician's Elephant,17 Mar 2023,PG,1h 39min,"Animation, Adventure, Comedy",English,N/A,53,6.6,"6,117",N/A
2,Mother's Day,24 May 2023,R,1h 34min,"Action, Drama, Thriller",Polish,N/A,N/A,5.3,"2,624",N/A
3,Assassin Club,16 May 2023,R,1h 51min,"Action, Thriller",English,N/A,27,4.7,"3,215",N/A
4,Carlos,29 Sep 2023,R,1h 27min,"Documentary, Biography",English,1 nomination,N/A,7.2,366,"$432,479"
5,The Devil on Trial,17 Oct 2023,R,1h 21min,"Documentary, Crime, Horror",English,N/A,N/A,5.6,"4,165",N/A
6,Trading Places,01 Mar 2023,R,1h 36min,Documentary,English,N/A,N/A,N/A,N/A,N/A
7,Sick Girl,20 Oct 2023,R,1h 39min,"Comedy, Drama",English,N/A,N/A,4.8,837,N/A
8,Killer Book Club,25 Aug 2023,R,1h 29min,"Horror, Thriller",Spanish,N/A,N/A,4.6,"5,051",N/A
9,Coffee Wars,21 Mar 2023,R,1h 44min,Comedy,English,N/A,N/A,4.6,"2,981",N/A


## 4. LƯU DỮ LIỆU THÔ VÀO FILE CSV

In [11]:
file_name = f"../Data/movie_data_part{str(part)}.csv"
df1.to_csv(file_name)